## Fetch code

In [1]:
!rm -rf YOLOv5_ByteTrack_Multithreading_TensorRT
!git clone https://github.com/xieincz/YOLOv5_ByteTrack_Multithreading_TensorRT.git

Cloning into 'YOLOv5_ByteTrack_Multithreading_TensorRT'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 305 (delta 14), reused 24 (delta 8), pack-reused 266
Receiving objects: 100% (305/305), 93.27 MiB | 18.28 MiB/s, done.
Resolving deltas: 100% (81/81), done.


## Prepare environment
note: Please turn on GPU

In [2]:
import torch
t=torch.tensor([1,2,3,4])
t.cuda()#Avoid colab prompt that GPU is not used

tensor([1, 2, 3, 4], device='cuda:0')

In [3]:
!wget https://developer.download.nvidia.com/compute/cuda/11.1.1/local_installers/cuda_11.1.1_455.32.00_linux.run

--2023-01-12 01:57:01--  https://developer.download.nvidia.com/compute/cuda/11.1.1/local_installers/cuda_11.1.1_455.32.00_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3532172082 (3.3G) [application/octet-stream]
Saving to: ‘cuda_11.1.1_455.32.00_linux.run’

cuda_11.1.1_455.32. 100%[===================>]   3.29G   233MB/s    in 14s     

2023-01-12 01:57:15 (240 MB/s) - ‘cuda_11.1.1_455.32.00_linux.run’ saved [3532172082/3532172082]



In [4]:
%%shell
sudo apt-get --purge remove "*cuda*" "*cublas*" "*cufft*" "*cufile*" "*curand*" \
 "*cusolver*" "*cusparse*" "*gds-tools*" "*npp*" "*nvjpeg*" "nsight*" "*nvvm*" --allow-change-held-packages -y

sudo apt-get autoremove

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'boinc-nvidia-cuda' for glob '*cuda*'
Note, selecting 'cuda-cuobjdump-12-0' for glob '*cuda*'
Note, selecting 'cuda-cusparse-cross-qnx-10-0' for glob '*cuda*'
Note, selecting 'cuda-toolkit-11-3-config-common' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-0' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-1' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-2' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-3' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-4' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-5' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-6' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-7' for glob '*cuda*'
Note, selecting 'cuda-nsight-compute-11-8' for glob '*cuda*'
Note, selecting 'cuda-nvprof-11-0' for glob '*cuda*'
Note, selecting 'cuda-nvprof-11-1' for glob '*cuda*'
Note, 

In [ ]:
!sudo sh cuda_11.1.1_455.32.00_linux.run --help

Options:
  --silent
    Performs an installation with no further user-input and minimal
    command-line output based on the options provided below. Silent
    installations are useful for scripting the installation of CUDA.
    Using this option implies acceptance of the EULA. The following flags
    can be used to customize the actions taken during installation. At
    least one of --driver, --uninstall, --toolkit, and --samples must
    be passed if running with non-root permissions.

  --driver
    Install the CUDA Driver.

  --toolkit
    Install the CUDA Toolkit.

  --toolkitpath=<path>
    Install the CUDA Toolkit to the <path> directory. If this flag is not
    provided, the default path of /usr/local/cuda-10.2 is used.

  --samples
    Install the CUDA Samples.

  --samplespath=<path>
    Install the CUDA Samples to the <path> directory. If this flag is not
    provided, the default path of /root/NVIDIA_CUDA-10.2_Samples is used.

  --librarypath=<path>
    Install libraries t

In [5]:
!sudo sh cuda_11.1.1_455.32.00_linux.run --silent  --toolkit --samples

In [6]:
!cat /var/log/cuda-installer.log

[INFO]: Setting silent=true
[INFO]: Setting toolkit=true
[INFO]: Setting samples=true
[INFO]: Driver not installed.
[INFO]: Checking compiler version...
[INFO]: gcc location: /usr/bin/gcc

[INFO]: gcc version: gcc version 7.5.0 (Ubuntu 7.5.0-3ubuntu1~18.04) 

[INFO]: Initializing menu
[INFO]: Silent install option: skipping driver
[INFO]: Components to install: 
[INFO]: CUDA Toolkit 11.1
[INFO]: Successfully created directory: /usr/local/cuda-11.1/
[INFO]: Successfully created directory: /var/log/nvidia/.uninstallManifests/
[INFO]: Created symlink: /usr/local/cuda -> /usr/local/cuda-11.1/
[INFO]: Successfully created directory: /usr/local/cuda-11.1/bin
[INFO]: Installed: /usr/local/cuda-11.1/bin/cuda-uninstaller
[INFO]: CUDA Tools 11.1
[INFO]: Successfully created directory: /var/log/nvidia/.uninstallManifests/CUDA_Toolkit_11.1-components/
[INFO]: CUDA Command Line Tools 11.1
[INFO]: Successfully created directory: /var/log/nvidia/.uninstallManifests/CUDA_Toolkit_11.1-components/CUDA_T

In [ ]:
!sudo apt-get --purge remove cuda nvidia* libnvidia-*
!sudo dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!sudo apt-get remove cuda-*
!sudo apt autoremove

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-headless-525-server' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'n

note: Next, the cell will ask you to enter options twice. Input 31 for the first time and 1 for the second time.

In [ ]:
!sudo apt-get -y install nvidia-driver-460
#31
#1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dkms dmsetup fakeroot keyboard-configuration libargon2-0 libcap2
  libcryptsetup12 libdevmapper1.02.1 libfakeroot libfontenc1 libidn11
  libip4tc0 libjansson4 libnss-systemd libnvidia-cfg1-470 libnvidia-common-470
  libnvidia-compute-470 libnvidia-decode-470 libnvidia-encode-470
  libnvidia-extra-470 libnvidia-fbc1-470 libnvidia-gl-470 libnvidia-ifr1-470
  libpam-systemd libpolkit-agent-1-0 libpolkit-backend-1-0
  libpolkit-gobject-1-0 libxfont2 libxkbfile1 libxnvctrl0 networkd-dispatcher
  nvidia-compute-utils-470 nvidia-dkms-470 nvidia-driver-470
  nvidia-kernel-common-470 nvidia-kernel-source-470 nvidia-prime
  nvidia-settings nvidia-utils-470 policykit-1 policykit-1-gnome python3-xkit
  screen-resolution-extra systemd systemd-sysv udev x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xserver-xorg-core-hwe-18.0

In [14]:
%%shell
cd /usr/local/cuda/samples/1_Utilities/deviceQuery
rm -f deviceQuery
sudo make
./deviceQuery

/usr/local/cuda/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_86,code=compute_86 -o deviceQuery deviceQuery.o 
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
mkdir -p ../../bin/x86_64/linux/release
cp deviceQuery ../../bin/x86_64/linux/release
./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          11.4 / 11.1
  CUDA Capa

In [8]:
%env LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:$LD_LIBARARY_PATH

env: LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:$LD_LIBARARY_PATH


In [9]:
%%shell

echo "install TensorRT 7.2.2.3 on home"
sudo apt update
sudo apt install -y nano

cd /home
echo "be careful! The package installed here is only applicable to Ubuntu 18.04 cuda 11.1 & 11.2"
echo "You can also download the file from the official website"

rm -f TensorRT7.2.2.3Ubuntu1804cuda11.1cudnn8.0.tar.gz

wget https://extremevision-js-userfile.oss-cn-hangzhou.aliyuncs.com/user-45129-files/594bdc1f-557d-40da-bf48-645e779b6808/TensorRT7.2.2.3Ubuntu1804cuda11.1cudnn8.0.tar.gz

tar -xzvf TensorRT7.2.2.3Ubuntu1804cuda11.1cudnn8.0.tar.gz

sudo echo -e "\nexport TENSORRT_ROOT=/home/TensorRT-7.2.2.3\n
export LD_LIBRARY_PATH=/home/TensorRT-7.2.2.3/lib:$LD_LIBRARY_PATH\n
export CUDA_INSTALL_DIR=/usr/local/cuda\n
export CUDNN_INSTALL_DIR=/usr/local/cuda\n
export LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:$LD_LIBARARY_PATH" >> ~/.bashrc

source ~/.bashrc

sudo echo -e "\nexport TENSORRT_ROOT=/home/TensorRT-7.2.2.3\n
export LD_LIBRARY_PATH=/home/TensorRT-7.2.2.3/lib:$LD_LIBRARY_PATH\n
export CUDA_INSTALL_DIR=/usr/local/cuda\n
export CUDNN_INSTALL_DIR=/usr/local/cuda\n
export LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:$LD_LIBARARY_PATH" >> /etc/profile

source /etc/profile

cd /home/TensorRT-7.2.2.3/python/
sudo pip3 install tensorrt-7.2.2.3-cp38-none-linux_x86_64.whl
cd ..
pip install uff/uff-0.6.9-py2.py3-none-any.whl
pip install graphsurgeon/graphsurgeon-0.4.5-py2.py3-none-any.whl

echo "install other packages"
sudo apt update
sudo apt-get install -y libeigen3-dev swig

install TensorRT 7.2.2.3 on home
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,552 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,348 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.dow

In [10]:
%env LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:/home/TensorRT-7.2.2.3/lib:$LD_LIBRARY_PATH
%env CUDA_INSTALL_DIR=/usr/local/cuda
%env CUDNN_INSTALL_DIR=/usr/local/cuda

env: LD_LIBRARY_PATH=/usr/local/cuda-11.1/targets/x86_64-linux/lib:/home/TensorRT-7.2.2.3/lib:$LD_LIBRARY_PATH
env: CUDA_INSTALL_DIR=/usr/local/cuda
env: CUDNN_INSTALL_DIR=/usr/local/cuda


In [15]:
%%shell

cd /home/TensorRT-7.2.2.3/data/mnist
python download_pgms.py

cd /home/TensorRT-7.2.2.3/samples/sampleMNIST
make
cd ../../bin
./sample_mnist


../Makefile.config:28: TRT_LIB_DIR is not specified, searching ../../lib, ../../lib, ../lib by default, use TRT_LIB_DIR=<trt_lib_directory> to change.
make: Nothing to be done for 'all'.
&&&& RUNNING TensorRT.sample_mnist # ./sample_mnist
[01/12/2023-02:16:40] [I] Building and running a GPU inference engine for MNIST
[01/12/2023-02:17:00] [I] [TRT] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[01/12/2023-02:17:00] [I] [TRT] Detected 1 inputs and 1 output network tensors.
[01/12/2023-02:17:00] [I] Input:
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@#- -#@@@@@@@@
@@@@@@@@@@@@@@#     @@@@@@@@
@@@@@@@@@@@@@#.     #@@@@@@@
@@@@@@@@@@@@#.   :*  +@@@@@@
@@@@@@@@@@@-      *: -@@@@@@
@@@@@@@@@@#   :+ .%* -@@@@@@
@@@@@@@@@#   :@*+@@@  #@@@@@
@@@@@@@@%-  .*@@@@@@  -@@@@@
@@@@@@@@:  #@%@@@@@@  :@@@@@
@@@@@@@#  #@@@@@@

In [ ]:
!cmake --version
# If version>=3.22.6, you can skip the next cell.

In [ ]:
%%shell

echo "install cmake 3.24.3 on home"
sudo apt-get update
sudo apt-get install -y build-essential libssl-dev

cd /home

echo "You can also download the file from the official website"
wget https://extremevision-js-userfile.oss-cn-hangzhou.aliyuncs.com/user-45129-files/496f99a0-2e7e-4454-8bad-f90dad285655/cmake-3.24.3.tar.gz

tar -zxvf cmake-3.24.3.tar.gz

cd cmake-3.24.3/
./bootstrap

make
sudo make install
hash -r

cmake --version

In [17]:
%%shell

echo "install opencv 4.1.2 on home"
cd /home

echo "You can also download the file from the official website"
wget https://extremevision-js-userfile.oss-cn-hangzhou.aliyuncs.com/user-45129-files/47d66834-779b-41d6-8393-8f6137d094a2/opencv412.zip
unzip opencv412.zip

find ./ -type f |xargs touch
cd opencv-4.1.2 && rm -rf build && mkdir build && cd build

cmake -D CMAKE_BUILD_TYPE=Release -D CMAKE_INSTALL_PREFIX=/usr/local -D OPENCV_EXTRA_MODULES_PATH=/home/opencv_contrib -D PYTHON_DEFAULT_EXECUTABLE=/usr/bin/python3 -D BUILD_opencv_python3=OFF -D BUILD_opencv_python2=OFF -D PYTHON3_EXCUTABLE=/usr/bin/python3 -D WITH_CUDA=OFF -D OPENCV_GENERATE_PKGCONFIG=ON ..

make

sudo make install

pkg-config --modversion opencv4

流式输出内容被截断，只能显示最后 5000 行内容。
   creating: opencv-4.1.2/modules/highgui/doc/
   creating: opencv-4.1.2/modules/highgui/doc/pics/
  inflating: opencv-4.1.2/modules/highgui/doc/pics/qtgui.png  
   creating: opencv-4.1.2/modules/highgui/include/
   creating: opencv-4.1.2/modules/highgui/include/opencv2/
   creating: opencv-4.1.2/modules/highgui/include/opencv2/highgui/
  inflating: opencv-4.1.2/modules/highgui/include/opencv2/highgui/highgui.hpp  
  inflating: opencv-4.1.2/modules/highgui/include/opencv2/highgui/highgui_c.h  
  inflating: opencv-4.1.2/modules/highgui/include/opencv2/highgui/highgui_winrt.hpp  
  inflating: opencv-4.1.2/modules/highgui/include/opencv2/highgui.hpp  
   creating: opencv-4.1.2/modules/highgui/misc/
   creating: opencv-4.1.2/modules/highgui/misc/java/
 extracting: opencv-4.1.2/modules/highgui/misc/java/filelist  
   creating: opencv-4.1.2/modules/highgui/misc/java/src/
   creating: opencv-4.1.2/modules/highgui/misc/java/src/java/
  inflating: opencv-4.1.2/modules

In [18]:
%%shell

echo "Compile the code of this project"
cd /content/YOLOv5_ByteTrack_Multithreading_TensorRT

cd yolov5_cpp_6
rm -rf build
mkdir build
cd build
cmake ..
make
cd ..
cd ..

cd yolobytedxc2_6

rm -rf CMakeFiles CMakeCache.txt Makefile cmake_install.cmake yolobyteapi.py libyolov5_trt.so

cmake ./ \
-DPYTHON_INCLUDE_DIR=$(python -c "from distutils.sysconfig import get_python_inc; print(get_python_inc())")  \
-DPYTHON_LIBRARY=$(python -c "import distutils.sysconfig as sysconfig; print(sysconfig.get_config_var('LIBDIR'))")

make 


Compile the code of this project
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found CUDA: /usr/local/cuda (found version "11.1") 
-- Found OpenCV: /usr/local (found version "4.1.2") 
-- Configuring done
-- Generating done
-- Buil

export tensorrt model

In [19]:
%%shell

echo -e "\nAttention! This is only design for online coding evnerviment.\n"
echo -e "\nIf you want to use it in local, please change the path in this file.\n"

model_dir=/content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/
src_dir=/content/YOLOv5_ByteTrack_Multithreading_TensorRT/
#最后面的斜杠不能少

model_type=m6
#可以是 n6 s6 m6 l6 x6之一

model_name=yolov5${model_type}
yolov5_cpp=yolov5_cpp_6
############################################################################################################
#/project/ev_sdk/model/yolov5l.pt -> /project/ev_sdk/model/yolov5l.engine
cp ${src_dir}${yolov5_cpp}/gen_wts.py ${src_dir}yolov5/
echo -e "\nConverting ${model_dir}${model_name}.pt to ${model_dir}${model_name}.wts\n"
python ${src_dir}yolov5/gen_wts.py -w ${model_dir}${model_name}.pt -o ${model_dir}${model_name}.wts
# update CLASS_NUM in yololayer.h if your model is trained on custom dataset
echo -e "\nAttention! Please update CLASS_NUM in ${src_dir}${yolov5_cpp}/yololayer.h if your model is trained on custom dataset.\n"
echo -e "\nConverting ${model_dir}${model_name}.wts to ${model_dir}${model_name}.engine\n"
${src_dir}${yolov5_cpp}/build/yolov5 -s ${model_dir}${model_name}.wts ${model_dir}${model_name}.engine ${model_type}
#rm ${model_dir}${model_name}.wts
echo -e "\nDone.\n"


Attention! This is only design for online coding evnerviment.


If you want to use it in local, please change the path in this file.


Converting /content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/yolov5m6.pt to /content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/yolov5m6.wts

YOLOv5 🚀 2023-1-12 Python-3.8.16 torch-1.13.1+cu116 CPU


Attention! Please update CLASS_NUM in /content/YOLOv5_ByteTrack_Multithreading_TensorRT/yolov5_cpp_6/yololayer.h if your model is trained on custom dataset.


Converting /content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/yolov5m6.wts to /content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/yolov5m6.engine

Loading weights: /content/YOLOv5_ByteTrack_Multithreading_TensorRT/cppmodels/yolov5m6.wts
Building engine, please wait for a while...
Build engine successfully!

Done.



## Demo

In [28]:
!python /content/YOLOv5_ByteTrack_Multithreading_TensorRT/demo.py

yolov5_trt_create  ... 
yolov5_trt_create  cuda engine... 
yolov5_trt_create  buffer ... 
yolov5_trt_create  stream ... 
yolov5_trt_create  done ... 
Init ByteTrack!
finish process video: /content/YOLOv5_ByteTrack_Multithreading_TensorRT/testdata/person_street.mp4
Init ByteTrack!
0
166.82264688899977


## Show the result

In [ ]:
import os

save_path = "/content/outputdir/output.mp4"
compressed_path = "/content/outputdir/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

In [ ]:
!pip install -U kora
from kora.drive import upload_public
from IPython.display import HTML

note: Next, you will be asked to authorize the login of Google Drive to upload videos. If you do not want to authorize, you can manually download the corresponding video file to play locally.

In [ ]:
url1 = upload_public('/content/YOLOv5_ByteTrack_Multithreading_TensorRT/testdata/person_street.mp4')
HTML(f"""<video src={url1} width=500 controls/>""")

In [ ]:
url2 = upload_public(compressed_path)
HTML(f"""<video src={url2} width=500 controls/>""")